# The Official AVM FRITZ!Box Interfaces

AVM offers developers two ways to communicate with their FRITZ!Box routers:
* **TR-064:** 
* **AHA-HTTP:** 

The `fritzconnection` library uses both of these interfaces, at least according to its documentation. I'd like to understand how this communication works. 

First things first, I need to get the login info.

In [127]:
import json

# load config file
CONFIG_FILE = "..\\..\\..\\_private_files\\sb4dfritz_secrets.ini"
with open(CONFIG_FILE,"r") as file:
    FRITZBOX = json.load(file)
# extract login data
FRITZ_IP = FRITZBOX['login']['ip']
FRITZ_USER = FRITZBOX['login']['user']
FRITZ_PWD = FRITZBOX['login']['pwd']
# extract device ains
DEVICES = FRITZBOX['ains']
AINS = list(DEVICES.values())


## TR-064

Here's a basic way to get the "current" power consumption of a FRITZ! smart plug. 
The procedure goes as follows:
* Formulate the request as a SOAP action
* Send it as a HTTP POST message using the `requests` module

### A concrete example


### What actions are there?


### Implemententing the actions


#### GetSpecificDeviceInfos [WORKS]

In [ ]:
import requests, warnings
from requests.auth import HTTPDigestAuth


def get_specific_device_info(device_ain:str)->requests.Response:
    """GetSpecificDeviceInfos action for TR-064 interfaces."""
    UPNP_URL = "https://" + FRITZ_IP + ":49443/upnp/control/x_homeauto"
    TR064_SERVICE = "urn:dslforum-org:service:X_AVM-DE_Homeauto:1"
    SOAP_ACTION = "GetSpecificDeviceInfos"
    # header for POST request
    request_headers = {
        'Content-Type': 'text/xml; charset="utf-8"', 
        'SoapAction': TR064_SERVICE + "#" + SOAP_ACTION
        # 'SoapAction': 'urn:dslforum-org:service:X_AVM-DE_Homeauto:1#GetSpecificDeviceInfos'
    }
    # data for POST request
    request_data = f"""
        <?xml version=\"1.0\"?> 
        <s:Envelope 
         xmlns:s=\"http://schemas.xmlsoap.org/soap/envelope/\" 
         s:encodingStyle=\"http://schemas.xmlsoap.org/soap/encoding/\"> 
            <s:Body> 
                <u:{SOAP_ACTION} xmlns:u=\"{TR064_SERVICE}\"> 
                    <NewAIN>{device_ain}</NewAIN> 
                </u:{SOAP_ACTION}> 
            </s:Body> 
        </s:Envelope>
        """
    # temporary ignore warnings (caused by self-signed certificate of FRITZ!Box)
    warnings.simplefilter('ignore')
    # send POST request
    request_result = requests.post(
        url=UPNP_URL, 
        auth=HTTPDigestAuth(FRITZ_USER, FRITZ_PWD), 
        headers=request_headers, 
        data=request_data, 
        verify=False
    )
    # allow warning again
    warnings.resetwarnings()
    return request_result

results = get_specific_device_info(AINS[0])
print(results.headers)
print(results.text)

{'Connection': 'keep-alive', 'Content-Length': '1677', 'Content-Type': 'text/xml; charset="utf-8"', 'Date': 'Sat, 26 Jul 2025 12:16:49 GMT', 'Server': 'FRITZ!Box 7490 UPnP/1.0 AVM FRITZ!Box 7490 113.07.60', 'Ext': ''}
<?xml version="1.0"?>
<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/" s:encodingStyle="http://schemas.xmlsoap.org/soap/encoding/">
<s:Body>
<u:GetSpecificDeviceInfosResponse xmlns:u="urn:dslforum-org:service:X_AVM-DE_Homeauto:1">
<NewDeviceId>16</NewDeviceId>
<NewFunctionBitMask>35712</NewFunctionBitMask>
<NewFirmwareVersion>04.27</NewFirmwareVersion>
<NewManufacturer>AVM</NewManufacturer>
<NewProductName>FRITZ!DECT 200</NewProductName>
<NewDeviceName>l&apos;angolo del caffè</NewDeviceName>
<NewPresent>CONNECTED</NewPresent>
<NewMultimeterIsEnabled>ENABLED</NewMultimeterIsEnabled>
<NewMultimeterIsValid>VALID</NewMultimeterIsValid>
<NewMultimeterPower>329</NewMultimeterPower>
<NewMultimeterEnergy>2879797</NewMultimeterEnergy>
<NewTemperatureIsEnabled>ENABLE


#### GetInfo [Works]

In [128]:
import requests, warnings
from requests.auth import HTTPDigestAuth

def get_info()->requests.Response:
    """GetSpecificDeviceInfos action for TR-064 interfaces."""
    UPNP_URL = "https://" + FRITZ_IP + ":49443/upnp/control/x_homeauto"
    TR064_SERVICE = "urn:dslforum-org:service:X_AVM-DE_Homeauto:1"
    SOAP_ACTION = "GetInfo"
    # header for POST request
    request_headers = {
        'Content-Type': 'text/xml; charset="utf-8"', 
        'SoapAction': TR064_SERVICE + "#" + SOAP_ACTION
        # 'SoapAction': 'urn:dslforum-org:service:X_AVM-DE_Homeauto:1#GetSpecificDeviceInfos'
    }
    # data for POST request
    request_data = f"""
        <?xml version=\"1.0\"?> 
        <s:Envelope 
         xmlns:s=\"http://schemas.xmlsoap.org/soap/envelope/\" 
         s:encodingStyle=\"http://schemas.xmlsoap.org/soap/encoding/\"> 
            <s:Body> 
                <u:{SOAP_ACTION} xmlns:u=\"{TR064_SERVICE}\"> 
                </u:{SOAP_ACTION}> 
            </s:Body> 
        </s:Envelope>
        """
    # temporary ignore warnings (caused by self-signed certificate of FRITZ!Box)
    warnings.simplefilter('ignore')
    # send POST request
    request_result = requests.post(
        url=UPNP_URL, 
        auth=HTTPDigestAuth(FRITZ_USER, FRITZ_PWD), 
        headers=request_headers, 
        data=request_data, 
        verify=False
    )
    # allow warning again
    warnings.resetwarnings()
    return request_result

results = get_info()
print(results.headers)
print(results.text)

{'Connection': 'keep-alive', 'Content-Length': '518', 'Content-Type': 'text/xml; charset="utf-8"', 'Date': 'Sat, 26 Jul 2025 12:16:47 GMT', 'Server': 'FRITZ!Box 7490 UPnP/1.0 AVM FRITZ!Box 7490 113.07.60', 'Ext': ''}
<?xml version="1.0"?>
<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/" s:encodingStyle="http://schemas.xmlsoap.org/soap/encoding/">
<s:Body>
<u:GetInfoResponse xmlns:u="urn:dslforum-org:service:X_AVM-DE_Homeauto:1">
<NewAllowedCharsAIN>0123456789ABCDEFabcdef :-grptmp</NewAllowedCharsAIN>
<NewMaxCharsAIN>19</NewMaxCharsAIN>
<NewMinCharsAIN>1</NewMinCharsAIN>
<NewMaxCharsDeviceName>79</NewMaxCharsDeviceName>
<NewMinCharsDeviceName>1</NewMinCharsDeviceName>
</u:GetInfoResponse>
</s:Body>
</s:Envelope>


#### GetGenericDeviceInfos [FAIL]

In [129]:
import requests, warnings
from requests.auth import HTTPDigestAuth

def get_generic_device_infos(device_index:str)->requests.Response:
    """GetSpecificDeviceInfos action for TR-064 interfaces."""
    UPNP_URL = "https://" + FRITZ_IP + ":49443/upnp/control/x_homeauto"
    TR064_SERVICE = "urn:dslforum-org:service:X_AVM-DE_Homeauto:1"
    SOAP_ACTION = "GetGenericDeviceInfos"
    # header for POST request
    request_headers = {
        'Content-Type': 'text/xml; charset="utf-8"', 
        'SoapAction': TR064_SERVICE + "#" + SOAP_ACTION
        # 'SoapAction': 'urn:dslforum-org:service:X_AVM-DE_Homeauto:1#GetSpecificDeviceInfos'
    }
    # data for POST request
    request_data = f"""
        <?xml version=\"1.0\"?> 
        <s:Envelope 
         xmlns:s=\"http://schemas.xmlsoap.org/soap/envelope/\" 
         s:encodingStyle=\"http://schemas.xmlsoap.org/soap/encoding/\"> 
            <s:Body> 
                <u:{SOAP_ACTION} xmlns:u=\"{TR064_SERVICE}\"> 
                    <NewIndex>{device_index}</NewIndex> 
                </u:{SOAP_ACTION}> 
            </s:Body> 
        </s:Envelope>
        """
    # temporary ignore warnings (caused by self-signed certificate of FRITZ!Box)
    warnings.simplefilter('ignore')
    # send POST request
    request_result = requests.post(
        url=UPNP_URL, 
        auth=HTTPDigestAuth(FRITZ_USER, FRITZ_PWD), 
        headers=request_headers, 
        data=request_data, 
        verify=False
    )
    # allow warning again
    warnings.resetwarnings()
    return request_result

results = get_generic_device_infos("16")
print(results.headers)
print(results.text)

{'Connection': 'keep-alive', 'Content-Length': '441', 'Content-Type': 'text/xml; charset="utf-8"', 'Ext': ''}
<?xml version="1.0"?>
<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/" s:encodingStyle="http://schemas.xmlsoap.org/soap/encoding/">
<s:Body>
<s:Fault>
<faultcode>s:Client</faultcode>
<faultstring>UPnPError</faultstring>
<detail>
<UPnPError xmlns="urn:dslforum-org:control-1-0">
<errorCode>713</errorCode>
<errorDescription>SpecifiedArrayIndexInvalid</errorDescription>
</UPnPError>
</detail>
</s:Fault>
</s:Body>
</s:Envelope>


#### SetSwitch


In [131]:
import requests, warnings
from requests.auth import HTTPDigestAuth


def set_switch(device_ain:str, target_state:str)->requests.Response:
    """GetSpecificDeviceInfos action for TR-064 interfaces."""
    UPNP_URL = "https://" + FRITZ_IP + ":49443/upnp/control/x_homeauto"
    TR064_SERVICE = "urn:dslforum-org:service:X_AVM-DE_Homeauto:1"
    SOAP_ACTION = "SetSwitch"

    # ALLOWED_STATES = ["ON", "OFF", "TOGGLE"]
    # if not target_state in ALLOWED_STATES:
    #     print("Target state must be 'ON', 'OFF', or 'TOGGLE'.")
    #     return
    # header for POST request
    request_headers = {
        'Content-Type': 'text/xml; charset="utf-8"', 
        'SoapAction': TR064_SERVICE + "#" + SOAP_ACTION
        # 'SoapAction': 'urn:dslforum-org:service:X_AVM-DE_Homeauto:1#GetSpecificDeviceInfos'
    }
    # data for POST request
    request_data = f"""
        <?xml version=\"1.0\"?> 
        <s:Envelope 
         xmlns:s=\"http://schemas.xmlsoap.org/soap/envelope/\" 
         s:encodingStyle=\"http://schemas.xmlsoap.org/soap/encoding/\"> 
            <s:Body> 
                <u:{SOAP_ACTION} xmlns:u=\"{TR064_SERVICE}\"> 
                    <NewAIN>{device_ain}</NewAIN> 
                    <NewSwitchState>{target_state}</NewSwitchState> 
                </u:{SOAP_ACTION}> 
            </s:Body> 
        </s:Envelope>
        """
    # temporary ignore warnings (caused by self-signed certificate of FRITZ!Box)
    warnings.simplefilter('ignore')
    # send POST request
    request_result = requests.post(
        url=UPNP_URL, 
        auth=HTTPDigestAuth(FRITZ_USER, FRITZ_PWD), 
        headers=request_headers, 
        data=request_data, 
        verify=False
    )
    # allow warning again
    warnings.resetwarnings()
    return request_result

results = set_switch(AINS[0], "ON")
print(results.headers)
print(results.text)


{'Connection': 'keep-alive', 'Content-Length': '278', 'Content-Type': 'text/xml; charset="utf-8"', 'Date': 'Sat, 26 Jul 2025 12:16:49 GMT', 'Server': 'FRITZ!Box 7490 UPnP/1.0 AVM FRITZ!Box 7490 113.07.60', 'Ext': ''}
<?xml version="1.0"?>
<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/" s:encodingStyle="http://schemas.xmlsoap.org/soap/encoding/">
<s:Body>
<u:SetSwitchResponse xmlns:u="urn:dslforum-org:service:X_AVM-DE_Homeauto:1"></u:SetSwitchResponse>
</s:Body>
</s:Envelope>


### Another go at `turn_off_when_idle`


#### Get current power

In [ ]:
from datetime import datetime, timezone

def get_current_power(ain):
    # request info from device keeping track of start and end times
    start = current_time()
    device_info = get_specific_device_info(ain)
    end = current_time()
    # extract power (in W) and timestamp 
    power = int((device_info.text).split("<NewMultimeterPower>")[1].split("</NewMultimeterPower>")[0]) / 100
    timestamp = parse_date_str(device_info.headers['Date'])
    # compute duration and latency
    duration = seconds_between(start, end)
    latency = seconds_between(timestamp, end)
    power_data = {
        "power":power,
        "time":timestamp,
        "start":start,
        "end":end,
        "duration":duration,
        "latency":latency,
    }
    return power_data

def current_time():
    return datetime.now()#.astimezone()

def seconds_between(start:datetime, end:datetime)->float:
    return (end - start).total_seconds()

def parse_date_str(date_str):
    date = datetime.strptime(date_str, "%a, %d %b %Y %H:%M:%S GMT")
    date = date.replace(tzinfo=timezone.utc)
    date = date.astimezone().replace(tzinfo=None)
    return date

get_current_power(AINS[0])

{'time': datetime.datetime(2025, 7, 26, 14, 52, 31),
 'power': 957.08,
 'start': datetime.datetime(2025, 7, 26, 14, 52, 29, 948824),
 'end': datetime.datetime(2025, 7, 26, 14, 52, 30, 891702),
 'duration': 0.942878,
 'latency': -0.108298}

#### Detect power measurement cycle

In [159]:
len({1,1})

1

In [ ]:
def detect_power_cycle(ain):
    two_last_measurements = []
    while True:
        measurement = get_current_power(ain)
        print(measurement)
        two_last_measurements.append(measurement)
        two_last_measurements = two_last_measurements[-2:]
        if len(two_last_measurements) >= 2:
            power_vals = set([data['power'] for data in two_last_measurements])
            if len(power_vals) == 2:
                break
    return two_last_measurements

detect_power_cycle(AINS[0])


        

{'time': datetime.datetime(2025, 7, 26, 15, 13, 51), 'power': 3.29, 'start': datetime.datetime(2025, 7, 26, 15, 13, 50, 619352), 'end': datetime.datetime(2025, 7, 26, 15, 13, 51, 829687), 'duration': 1.210335, 'latency': 0.829687}
{'time': datetime.datetime(2025, 7, 26, 15, 13, 52), 'power': 3.29, 'start': datetime.datetime(2025, 7, 26, 15, 13, 51, 830728), 'end': datetime.datetime(2025, 7, 26, 15, 13, 52, 736828), 'duration': 0.9061, 'latency': 0.736828}
{'time': datetime.datetime(2025, 7, 26, 15, 13, 53), 'power': 3.29, 'start': datetime.datetime(2025, 7, 26, 15, 13, 52, 737946), 'end': datetime.datetime(2025, 7, 26, 15, 13, 53, 652541), 'duration': 0.914595, 'latency': 0.652541}
{'time': datetime.datetime(2025, 7, 26, 15, 13, 54), 'power': 3.29, 'start': datetime.datetime(2025, 7, 26, 15, 13, 53, 653590), 'end': datetime.datetime(2025, 7, 26, 15, 13, 54, 592605), 'duration': 0.939015, 'latency': 0.592605}
{'time': datetime.datetime(2025, 7, 26, 15, 13, 55), 'power': 3.29, 'start': d

### Random Notes
Note that this function returns the response as an instance of the `requests.Response` class.
While I'm at it, I might just as well take a quick look at what this class does.

In [132]:
# print documentation of the `requests.Response` class
print(help(results))

Help on Response in module requests.models object:

class Response(builtins.object)
 |  The :class:`Response <Response>` object, which contains a
 |  server's response to an HTTP request.
 |
 |  Methods defined here:
 |
 |  __bool__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |
 |      This attribute checks if the status code of the response is between
 |      400 and 600 to see if there was a client error or a server error. If
 |      the status code, is between 200 and 400, this will return True. This
 |      is **not** a check to see if the response code is ``200 OK``.
 |
 |  __enter__(self)
 |
 |  __exit__(self, *args)
 |
 |  __getstate__(self)
 |      Helper for pickle.
 |
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  __iter__(self)
 |      Allows you to use a response as an iterator.
 |
 |  __nonzero__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |
 |      This attribute checks if 

The relevant information is found in the attributes `.headers` and `.text`. 
* `.headers` is a type of dictionary containing metadata of the response including a time stamp with second precision
* `.text` is a string containing the XML encoded response

In [133]:
# print data types
print("Type of .headers:", type(results.headers))
print("Type of .text:   ", type(results.text))
print("-"*80)
# print dictionary keys
print("Keys in results.headers:")
for key, val in results.headers.items():
    print(f"- {key:14s} : {val}")


Type of .headers: <class 'requests.structures.CaseInsensitiveDict'>
Type of .text:    <class 'str'>
--------------------------------------------------------------------------------
Keys in results.headers:
- Connection     : keep-alive
- Content-Length : 278
- Content-Type   : text/xml; charset="utf-8"
- Date           : Sat, 26 Jul 2025 12:16:49 GMT
- Server         : FRITZ!Box 7490 UPnP/1.0 AVM FRITZ!Box 7490 113.07.60
- Ext            : 


In [134]:
print("Type of .text:   ", type(results.text))
print("-"*80)
# print results.text
print("Contents of results.text:\n")
print(results.text)

Type of .text:    <class 'str'>
--------------------------------------------------------------------------------
Contents of results.text:

<?xml version="1.0"?>
<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/" s:encodingStyle="http://schemas.xmlsoap.org/soap/encoding/">
<s:Body>
<u:SetSwitchResponse xmlns:u="urn:dslforum-org:service:X_AVM-DE_Homeauto:1"></u:SetSwitchResponse>
</s:Body>
</s:Envelope>


In [135]:

def get_power(device_ain):
    """Get the current power consumption using the TR-064 interfaces."""
    # get specific device information
    request_result = get_specific_device_info(device_ain)
    # extract power value and date
    power = int((request_result.text).split("<NewMultimeterPower>")[1].split("</NewMultimeterPower>")[0]) / 100
    date = request_result.headers['Date']
    return power, date
    # return request_result

for dev, ain in DEVICES.items():
    print(dev, get_power(ain))

angolo (3.29, 'Sat, 26 Jul 2025 12:16:50 GMT')
tv (86.97, 'Sat, 26 Jul 2025 12:16:51 GMT')
